## 降雨侵蚀力因子计算

In [1]:
import geemap
import ee

Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [2]:
import os
os.chdir(r'D:\Work\naqu')

In [3]:
shp = geemap.shp_to_ee('那曲市矢量/那曲市WGS_1984.shp')
roi = shp.geometry()
Map.center_object(roi, 7)
Map.addLayer(roi, {}, 'roi')

In [4]:
def daily_mask(img):
    img = img.mask(img.gte(12))
    return img

def daily_where(img):
    img = img.where(img.lt(12), 0)
    return img

def clip_img(img):
    return img.clip(roi)

### Pd12计算


In [5]:
images = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterDate('1985', '2022').map(clip_img)
images_count = images.map(daily_mask).count()
images_count = images_count.where(images_count.eq(0), 1)
images_sum = images.map(daily_where).sum()

Pd12 = images_sum.divide(images_count)
Map.addLayer(Pd12, {'min': 0, 'max': 50}, 'Pd12')

### Py12计算

In [6]:
year_images = []
for year in range(1985, 2022):
    images_year = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterDate(f'{year}', f'{year+1}').map(clip_img)
    images_sum_year = images.map(daily_where).sum()
    year_images.append(images_sum_year)
    
Py12 = ee.ImageCollection(year_images).mean()
# Map.addLayer(Py12, {'min': 0, 'max': 2000}, 'Py12')

### α、β计算

In [7]:
B = Py12.expression("0.8363 + 18.144 / Pd12 + 24.455 / Py12", 
                    {'Pd12': Pd12, 'Py12': Py12})

A = B.expression("21.586 * B ** -7.1891", {'B': B})

In [8]:
def calc_daily_r(img):
    img = img.expression("img ** B", {'img': img, 'B': B})
    return img

### 多年年平均降雨侵蚀力计算

In [9]:
Rs = []
for year in range(1985, 2022):
    prcp_year = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterDate(f'{year}', f'{year+1}')
    R_year = prcp_year.map(clip_img).map(daily_where).map(calc_daily_r).sum().multiply(A)
    Rs.append(R_year)
R = ee.ImageCollection(Rs).mean()
# Map.addLayer(R, {'min': 0, 'max': 5000}, 'R')

In [10]:
geemap.download_ee_image(R, 'R.tif', roi, 'epsg:3857', scale=4000)

R.tif: |                                                               | 0.00/487k (raw) [  0.0%] in 00:00 (et…

There is no STAC entry for: None
